In [ ]:
# default_exp files

# files
> Easily upload, download, and manage files on Server.

In [ ]:
# export
import requests

from yx_motor.api import API


class Files:
    "Class for handling AAH files endpoints."

    def __init__(self, api: API):
        self.api = api
        self.base_endpoint = "files/"
        self.valid_conflict_actions = ["MERGE", "CREATE_COPY"]

    def download_file(self, file_uuid: str, download_path: str, version: int = None):
        response = self.api.get(
            url=f"{self.base_endpoint}content",
            params={"id": file_uuid, "version": version},
        )
        with open(download_path, "wb") as f:
            f.write(response.content)
        return response

    def upload_file(
        self,
        filename: str,
        upload_path: str,
        description: str = None,
        conflict_action: str = "CREATE_COPY",
    ):
        if conflict_action:
            self.validate_conflict_action(conflict_action)

        upload_headers = {
            "Content-Type": "application/json",
            "Accept": "*/*",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip,deflate",
            "path": upload_path,
            "description": description,
            "conflict_action": conflict_action,
        }

        with open(filename, "rb") as f:
            blob = f.read()
        response = self.api.post(
            url=f"{self.base_endpoint}", data=blob, non_default_headers=upload_headers
        )
        return response

    def update_file(self):
        # This seems to update metainfo, may not be needed for MVP
        # TODO: MVP
        pass

    def get_file_versions(self, file_uuid: str):
        response = self.api.get(
            url=f"{self.base_endpoint}versions/", params={"fileUuid": file_uuid}
        )
        return response

    def delete_file(self, asset_path: str, hard=False):
        payload = {"assetPaths": [f"{asset_path}"]}

        if hard == True:
            targeturl = "remove"
        else:
            targeturl = "softDelete"

        response = self.api.post(url=f"{self.base_endpoint}{targeturl}", json=payload)

        return response

    def move_file(
        self,
        source_path: str,
        target_path: str,
        move_type="moveFiles",
        versions_action="ALL_VERSIONS",
        conflicts_action="SKIP",
    ):
        # TODO discuss move_type, move or copy
        payload = {
            "assets": [
                {"sourcePath": f"{source_path}", "targetPath": f"{target_path}"}
            ],
            "versionsAction": f"{versions_action}",
            "conflictsAction": f"{conflicts_action}",
        }

        response = self.api.post(url=f"{self.base_endpoint}{move_type}", json=payload)
        return response

    def restore_deleted_file(self, asset_path: str = None, asset_id: str = None):
        asset_paths = {}
        asset_ids = {}
        if asset_path:
            asset_paths = {"assetPaths": [f"{asset_path}"]}
        if asset_id:
            asset_ids = {"assetIds": [f"{asset_id}"]}
        payload = {**asset_paths, **asset_ids}
        # Does onlyDescendants have other values besides True?
        response = self.api.post(
            url=f"{self.base_endpoint}restoreDeleted", json=payload
        )
        return response

    def validate_conflict_action(self, conflict_action: str):
        if conflict_action not in self.valid_conflict_actions:
            raise ValueError(
                f"Specified conflict action must be one of {self.valid_conflict_actions}"
            )
        else:
            pass

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 01_client.ipynb.
Converted 02_jobs.ipynb.
Converted 03_api.ipynb.
Converted 04_authenticate.ipynb.
Converted 05_files.ipynb.
Converted 06_workflows.ipynb.
Converted index.ipynb.


In [ ]:
#hide
from nbdev.sync import script2notebook
#script2notebook()

In [ ]:
#hide
# just removing the insecure warning for now
# TODO: Secure requests and remove this code
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
from yx_motor.client import Client

In [ ]:
#hide
from private import server_vars

In [ ]:
#hide
base_url=server_vars['base_url']
login_email = server_vars['login_email']
login_pwd = server_vars['login_pwd']

In [ ]:
#hide
dev_client = Client(base_url,
              login_email, 
              login_pwd)

In [ ]:
dev_client.api.is_authenticated
files = Files(dev_client.api)

In [ ]:
response = files.download_file("68efa737-d9e0-4a91-8379-737062ddc158","test.log")

In [ ]:
response.content

b'-|worker|Retrieved file example.yxzp\r\n-|worker|Engine run started\r\n2|1|Autodetected Code Page: UTF-8\r\n2|8|input_one.csv|4 records were read from "input_one.csv"\r\n3|1|Autodetected Code Page: UTF-8\r\n3|8|input_two.csv|1 records were read from "input_two.csv" (ended by a downstream tool)\r\n-|worker|Engine run finished in 0.47 seconds'

In [ ]:
#hide
from nbdev.sync import script2notebook
#script2notebook()

In [ ]:
#hide

# Integration Test Scaffolding

# Seed the server.  Let's put code in this cell which seeds the server with files.
"""
1. Upload several files to predetermined path on the VFS.  Extract the UUIDS from the responses,
put

TODO: Finish this.
TODO: Pathlib
"""

# just removing the insecure warning for now
# TODO: Secure requests and remove this code
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from yx_motor.client import Client
from private import server_vars
base_url=server_vars['base_url']
login_email = server_vars['login_email']
login_pwd = server_vars['login_pwd']
example_client = Client(base_url,
                  login_email, 
                  login_pwd)

files = Files(example_client.api)

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/authenticate/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Content-Length': '60'}
Response Status: 200


In [ ]:
#hide
#upload some workflow files one at a time.
#then, set UUID = response from api
workflow_A_name = "WorkflowA.yxmd"
workflow_B_name = "WorkflowB.yxmd"
workflow_A = files.upload_file(workflow_A_name,upload_path=f"/Workspaces/Public/{workflow_A_name}",conflict_action="MERGE").json()
workflow_B = files.upload_file(workflow_B_name,upload_path=f"/Workspaces/Public/{workflow_B_name}",conflict_action="MERGE").json()


POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'path': '/Workspaces/Public/WorkflowA.yxmd', 'conflict_action': 'MERGE', 'Cookie': 'ayxSession=s%3A30a232f4-e4db-4f70-9ab3-d7a04b073900.wxwyym3%2B6SNSTXSsMQa0zYFElStEErbBHUQgW8sGKDY', 'Content-Length': '3250'}
Response Status: 200
POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'path': '/Workspaces/Public/WorkflowB.yxmd', 'conflict_action': 'MERGE', 'Cookie': 'ayxSession=s%3A30a232f4-e4db-4f70-9ab3-d7a04b073900.wxwyym3%2B6SNSTXSsMQa0zYFElStEErbBHUQgW8sGKDY', 'Content-Length': '4183'}
Respo

In [ ]:
#hide
#set uuid and path values for more testing
workflow_A_uuid = workflow_A['uuid']
workflow_A_path = workflow_A['path']
workflow_B_uuid = workflow_B['uuid']
workflow_B_path = workflow_B['path']


## Examples

**Note**:
The files commands are intended to be called by users through the Client class.  Below, you will see that to accommodate this, we are creating the files object the same way the Client class does, by passing in an authenticated API object.  

In normal practice, a user will simply create a client object, and be able to call `client.files.download_file('xxxxxx-xxxxx')`.

However, for this documentation, we are calling the files object directly to better demonstrate the Files class in an isolated fashion.

### Download Files Example

In [ ]:
from nbdev.showdoc import *
show_doc(Files.download_file)

<h4 id="Files.download_file" class="doc_header"><code>Files.download_file</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.download_file</code>(**`file_uuid`**:`str`, **`download_path`**:`str`, **`version`**:`int`=*`None`*)



The download_file method allows a user to pass in the unique identifier for the file, along with the path they wish to download the file to on the client machine.  The file will be downloaded and saved to the path specified.

**Arguments**:
- file_uuid: The VFS unique identifier for the file the user wishes to download.
- download_path: The path on the user's local file system which they wish the file to be persisted to once it is downloaded.
- version: Optional.  The version of the VFS asset the user wishes to donwload.  Defaults to latest version if not specified.  TODO: Need to double check this statement.

In [ ]:
# Initialize the files object the same way the Client object does.
files = Files(example_client.api)

Below is the unique identifier for the file we wish to download from the Alteryx Analytics Hub's Virtual File System (VFS).

In [ ]:
asset_id = workflow_A_uuid

Next, we will trigger the method.  To better illustrate what the download_file method does, we are going to capture the response.  It is important to note that this is not necessary.  Simply calling the `files.download_file` command is all that is necessary to successfully download a file to the specified path.

First, let's show the contents of the directory we wish to download the file to.

In [ ]:
!mkdir example_downloads

In [ ]:
!dir example_downloads

 Volume in drive C is Windows
 Volume Serial Number is 241B-A66E

 Directory of C:\Users\jesse.clark\Documents\GitHub\yx_motor\example_downloads

04/28/2020  12:24 PM    <DIR>          .
04/28/2020  12:24 PM    <DIR>          ..
               0 File(s)              0 bytes
               2 Dir(s)  689,385,291,776 bytes free


In [ ]:
#hide
!del example_downloads

As we can see, the directory is currently empty.

Next, let's trigger the download_file command.

In [ ]:
response = files.download_file(file_uuid=asset_id,
                               download_path=f"example_downloads/{workflow_A_name}")

GET sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/content?id=a836329c-2ea4-40c0-968e-275b43ff00d7
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A17f3e051-7832-4e0e-ab4e-184cbb835e98.%2BhT5ggwd2RXV5BDot3q93ert5f2kXqM02CPfZtpsjnU'}
Response Status: 200


Now let's check the same directory from above.  We will inspect the response object after this.

In [ ]:
!dir example_downloads

 Volume in drive C is Windows
 Volume Serial Number is 241B-A66E

 Directory of C:\Users\jesse.clark\Documents\GitHub\yx_motor\example_downloads

04/28/2020  12:26 PM    <DIR>          .
04/28/2020  12:26 PM    <DIR>          ..
04/28/2020  12:26 PM             3,250 WorkflowA.yxmd
               1 File(s)          3,250 bytes
               2 Dir(s)  689,396,101,120 bytes free


As you can see, the workflow has been downloaded to the specified location.

Now, let's inspect the response object.

In [ ]:
response.content # The requested file is attached to the response as a binary object.

b'<?xml version="1.0"?>\n<AlteryxDocument yxmdVer="2019.1">\n  <Nodes>\n    <Node ToolID="2">\n      <GuiSettings Plugin="AlteryxBasePluginsGui.GenerateRows.GenerateRows">\n        <Position x="318" y="258" />\n      </GuiSettings>\n      <Properties>\n        <Configuration>\n          <UpdateField value="False" />\n          <UpdateField_Name />\n          <CreateField_Name>RowCount</CreateField_Name>\n          <CreateField_Type>Int32</CreateField_Type>\n          <CreateField_Size>4</CreateField_Size>\n          <Expression_Init>1</Expression_Init>\n          <Expression_Cond>RowCount &lt;= 100000000</Expression_Cond>\n          <Expression_Loop>RowCount + 1</Expression_Loop>\n        </Configuration>\n        <Annotation DisplayMode="0">\n          <Name />\n          <DefaultAnnotationText />\n          <Left value="False" />\n        </Annotation>\n      </Properties>\n      <EngineSettings EngineDll="AlteryxBasePluginsEngine.dll" EngineDllEntryPoint="AlteryxGenerateRows" />\n  

Here, we see that the binary object is indeed an Alteryx workflow.

### Upload Files Example

In [ ]:
from nbdev.showdoc import *
show_doc(Files.upload_file)

<h4 id="Files.upload_file" class="doc_header"><code>Files.upload_file</code><a href="__main__.py#L24" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.upload_file</code>(**`filename`**:`str`, **`upload_path`**:`str`, **`description`**:`str`=*`None`*, **`conflict_action`**:`str`=*`'CREATE_COPY'`*)



The upload_file method allows a user to upload a file from their local filesytem to the Alteryx Analytics Hub's Virtual File System (VFS).

TODO: Need to rename the filename argument to file_path to avoid confusion.

**Arguments**:
- filename: Path to the file on the user's local file system
- upload_path: The path on the AAH VFS the user wishes the file to be uploaded to.
- description: Optional.  Description the user wants to be attached to the file in the VFS.
- conflict_action: Defaults to 'CREATE_COPY'.  One of 'CREATE_COPY'| 'MERGE'
    - CREATE_COPY: If file already exists at specified path, a copy is created.
    - MERGE: If file already exists, new version is created at same location.


In [ ]:
# Initialize the files object the same way the Client object does.
files = Files(example_client.api)

#### Simple Upload Example

Let's upload a yxzp file in our local filesystem to a location on Alteryx Analytics Hub's VFS, specified via the upload_path parameter.

In [ ]:
response = files.upload_file(filename=r"test.yxzp",
                             upload_path="/Workspaces/Public/example_1.yxzp")

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'path': '/Workspaces/Public/example_1.yxzp', 'conflict_action': 'CREATE_COPY', 'Cookie': 'ayxSession=s%3Aa4e6c322-5a8d-4909-ad85-0f72de9e19cf.jbxJCEZNLq3mmxW0RWJbCiayPjQvxEj0sP2moTAkFZs', 'Content-Length': '2176'}
Response Status: 200


In [ ]:
response.json()

{'fileName': 'example_1.yxzp',
 'extension': 'yxzp',
 'inherits': True,
 'isHidden': False,
 'onlyOwnerShares': False,
 'path': '/Workspaces/Public/example_1.yxzp',
 'folderType': None,
 'version': 1,
 'created': '2020-04-28T15:56:36.357Z',
 'versionCreated': '2020-04-28T15:56:36.357Z',
 'maxVersion': 1,
 'links': None,
 'location': '',
 'uuid': '51cc3f05-2bed-4854-83e4-905465af9957',
 'metaHash': 'd214d65ab55a2a1c73de7c90fcc1aecfc28dd88dbec71cdeaeb3ffb1ef170053',
 'metadata': {'yxType': 'WORKFLOW_APP'},
 'contentHash': 'b7960c08c75e5e4e2bcf5fa32582eee75a07560554c6993c603759a8a2ce2f5f',
 'contentId': '6925a8c7-09f8-4dea-b365-b117130d597a',
 'contentSize': 2176,
 'entryOwner': {'avatar': None,
  'email': 'siteadmin3@example.com',
  'firstName': 'Seeded',
  'id': '0625e3e2-10fb-4df6-99c6-4e38712c9c36',
  'lastName': 'siteadmin3',
  'name': 'Seeded siteadmin3',
  'userName': 'siteadmin3'},
 'md5Hash': '1316e622c535fcfef109f75484910f38',
 'assetCategory': 'WORKFLOW_APP',
 'onlySiteAdminSha

Above, we can see that the response shows the file was successfully uploaded to the upload path we specified.  

#### Upload with duplicate file example

What happens if we upload the same file twice?  Let's run the same line of code a second time to see.

In [ ]:
response = files.upload_file(filename=r"test.yxzp",
                             upload_path="/Workspaces/Public/example_1.yxzp")

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'path': '/Workspaces/Public/example_1.yxzp', 'conflict_action': 'CREATE_COPY', 'Cookie': 'ayxSession=s%3A9da4a129-5363-4127-8554-8a7c54202951.bPxpXPlo4zYsmjVBXhKuQ%2BYu2n5KFpD36z7pkSrbjW4', 'Content-Length': '2176'}
Response Status: 200


In [ ]:
response.json()

{'fileName': 'example_128129.yxzp',
 'extension': 'yxzp',
 'inherits': True,
 'isHidden': False,
 'onlyOwnerShares': False,
 'path': '/Workspaces/Public/example_128129.yxzp',
 'folderType': None,
 'version': 1,
 'created': '2020-04-27T18:42:54.938Z',
 'versionCreated': '2020-04-27T18:42:54.938Z',
 'maxVersion': 1,
 'links': None,
 'location': '',
 'uuid': '314a8283-0bb7-43b8-84b7-70d5fa734c6f',
 'metaHash': 'd214d65ab55a2a1c73de7c90fcc1aecfc28dd88dbec71cdeaeb3ffb1ef170053',
 'metadata': {'yxType': 'WORKFLOW_APP'},
 'contentHash': 'b7960c08c75e5e4e2bcf5fa32582eee75a07560554c6993c603759a8a2ce2f5f',
 'contentId': '52d6897a-a07c-47aa-a66e-e85dfb2f6399',
 'contentSize': 2176,
 'entryOwner': {'avatar': None,
  'email': 'siteadmin3@example.com',
  'firstName': 'Seeded',
  'id': '977d6d2d-a6be-4533-9d29-c8936138b833',
  'lastName': 'siteadmin3',
  'name': 'Seeded siteadmin3',
  'userName': 'siteadmin3'},
 'md5Hash': '1316e622c535fcfef109f75484910f38',
 'assetCategory': 'WORKFLOW_APP',
 'onlySi

If you look above, you see that a copy of the file was created in the same location, but now the file is named example_1(1).yxzp.  

NOTE: It WAS named that in the payload.  Update at noon on 4/27/20 seems to have broken this, so that the UI shows the example_1(1) name, but the vfs path appears to now have a uid appended to it.

This is because the default conflict action is 'CREATE_COPY'.  

#### Upload a file with MERGE command

Let's now repeat the call, but this time let's specify 'MERGE' as the conflict action.  Also, we're changing the source of the file to a larger one, to ensure that the contentSize property in the JSON response proves that the file has been updated.

In [ ]:
response = files.upload_file(filename=r"czech_devops.yxmd",
                             upload_path="/Workspaces/Public/example_1.yxzp",
                             conflict_action='MERGE')

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'path': '/Workspaces/Public/example_1.yxzp', 'conflict_action': 'MERGE', 'Cookie': 'ayxSession=s%3A9da4a129-5363-4127-8554-8a7c54202951.bPxpXPlo4zYsmjVBXhKuQ%2BYu2n5KFpD36z7pkSrbjW4', 'Content-Length': '4055'}
Response Status: 200


In [ ]:
response.json()

{'fileName': 'example_1.yxzp',
 'extension': 'yxzp',
 'inherits': True,
 'isHidden': False,
 'onlyOwnerShares': False,
 'path': '/Workspaces/Public/example_1.yxzp',
 'folderType': None,
 'version': 2,
 'created': '2020-04-27T18:42:21.595Z',
 'versionCreated': '2020-04-27T22:44:26.724Z',
 'maxVersion': 2,
 'links': None,
 'location': '',
 'uuid': 'd0c39ee9-4725-49ed-966b-c03dbf33005d',
 'metaHash': '105263d9a1f6534fbbbb9abed4c9b7815c145c6792fcf66df7bb5576c6474af1',
 'metadata': {'tools': [],
  'yxType': 'WORKFLOW',
  'vfsInputs': {},
  'workflowInfo': {'isE2': False, 'modules': ['Magic-8-Ball.yxmd']},
  'packageAssets': ['Magic-8-Ball.yxmd']},
 'contentHash': 'c1d63578a41c19764af294616acac0f84c6aa40781496b643ebcbf644b79b642',
 'contentId': 'b07087f3-3c39-4349-88f0-363f94afa570',
 'contentSize': 4055,
 'entryOwner': {'avatar': None,
  'email': 'siteadmin3@example.com',
  'firstName': 'Seeded',
  'id': '977d6d2d-a6be-4533-9d29-c8936138b833',
  'lastName': 'siteadmin3',
  'name': 'Seeded s

In the payload above, you can see that the version of the file is now 2, and that it's contentSize property is now 4055, bigger than the original 2176 value.

This demonstrates that the file has indeed been updated in place, with a versioned copy.  

This MERGE command has a very powerful property:

It uses a "content hash" on the file being uploaded to detect if the file has actually changed.  If we run the above command again, with the same file, we will see that the file's version remains the same.

In [ ]:
# Extract the file uuid from the response to demonstrate the get versions module later.
uploaded_file_uuid = response.json()['uuid']

In [ ]:
from nbdev.showdoc import *
show_doc(Files.get_file_versions)

<h4 id="Files.get_file_versions" class="doc_header"><code>Files.get_file_versions</code><a href="__main__.py#L56" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.get_file_versions</code>(**`file_uuid`**:`str`)



The get_file_versions method returns a list of all of the versions for a given VFS asset.

**Arguments**:
- file_uuid: VFS Asset unique identifier for the VFS file you want to get versions for.

In [ ]:
# Initialize the files object the same way the Client object does.
files = Files(example_client.api)

In [ ]:
response = files.get_file_versions(file_uuid=uploaded_file_uuid)

GET sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/versions/?fileUuid=d0c39ee9-4725-49ed-966b-c03dbf33005d
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A9da4a129-5363-4127-8554-8a7c54202951.bPxpXPlo4zYsmjVBXhKuQ%2BYu2n5KFpD36z7pkSrbjW4'}
Response Status: 200


In [ ]:
response.json()

[{'version': 2,
  'metadata': {'tools': [],
   'yxType': 'WORKFLOW',
   'vfsInputs': {},
   'workflowInfo': {'isE2': False, 'modules': ['Magic-8-Ball.yxmd']},
   'packageAssets': ['Magic-8-Ball.yxmd']},
  'comment': None,
  'creator': {'avatar': None,
   'email': 'siteadmin3@example.com',
   'firstName': 'Seeded',
   'id': '977d6d2d-a6be-4533-9d29-c8936138b833',
   'lastName': 'siteadmin3',
   'name': 'Seeded siteadmin3'},
  'owner': {'avatar': None,
   'email': 'siteadmin3@example.com',
   'firstName': 'Seeded',
   'id': '977d6d2d-a6be-4533-9d29-c8936138b833',
   'lastName': 'siteadmin3',
   'name': 'Seeded siteadmin3'},
  'createdAt': '2020-04-27T22:44:26.724Z'},
 {'version': 1,
  'metadata': {'tools': [],
   'yxType': 'WORKFLOW',
   'vfsInputs': {},
   'workflowInfo': {'isE2': False, 'modules': ['Magic-8-Ball.yxmd']},
   'packageAssets': ['Magic-8-Ball.yxmd']},
  'comment': None,
  'creator': {'avatar': None,
   'email': 'siteadmin3@example.com',
   'firstName': 'Seeded',
   'id': '

In [ ]:
from nbdev.showdoc import *
show_doc(Files.delete_file)

<h4 id="Files.delete_file" class="doc_header"><code>Files.delete_file</code><a href="__main__.py#L62" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.delete_file</code>(**`asset_path`**:`str`, **`hard`**=*`False`*)



The delete_file method deletes a given asset located at the specified asset path in the VFS. Optionally, the user can specify whether the delete is permanent (hard=True) or reversable (hard=False).  

**Arguments**:
- asset_path: The VFS path of the asset the user wants to delete.
- hard: (Optional, defaults to False) If true, specifies a hard (permanent) delete. If false, the deleted asset can be restored.

In [ ]:
# Initialize the files object the same way the Client object does.
files = Files(example_client.api)

In [ ]:
response = files.delete_file(asset_path=r'/Workspaces/Public/example_1.yxzp', 
                             hard=True)

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/remove
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3Aa4e6c322-5a8d-4909-ad85-0f72de9e19cf.jbxJCEZNLq3mmxW0RWJbCiayPjQvxEj0sP2moTAkFZs', 'Content-Length': '53'}
Response Status: 401


In [ ]:
response.json()

{'errors': [{'code': 'USR_SESSION_INVALID', 'message': 'Session is invalid'}]}

In [ ]:
from nbdev.showdoc import *
show_doc(Files.move_file)

<h4 id="Files.move_file" class="doc_header"><code>Files.move_file</code><a href="__main__.py#L74" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.move_file</code>(**`source_path`**:`str`, **`target_path`**:`str`, **`move_type`**=*`'moveFiles'`*, **`versions_action`**=*`'ALL_VERSIONS'`*, **`conflicts_action`**=*`'SKIP'`*)



The move_file method moves a given asset from the source_path in the VFS to the target_path. Optionally, the user can specify the move_type, versions_action, or conflicts_action.

**Arguments**:
- source_path: The VFS path of the asset the user wants to move.
- target_path: The destination path in the VFS where the user wants the file moved to.
- move_type: (optional, defaults to moveFiles) If 'moveFiles', moves the file from source_path to target_path. If 'copy', creates a copy of the file in the target_path
- versions_action: (optional, defaults to ALL_VERSIONS) specifies if version history should be moved along with the file
- conflicts_action: (optional, defaults to SKIP) specifies what should happen if a file with the same name already exists in the target_path. options are SKIP (no action taken), FAIL_OPERATION (return an error), MERGE (update the file in target_path with the new file from source_path).

In [ ]:
# Initialize the files object the same way the Client object does.
files = Files(example_client.api)

### Simple move example

Lets try using a move command to rename a file

In [ ]:
#hide
#create an initial file to move
move_file_name="move_file_example.yxmd"
move_file_path=f"/Workspaces/Public/{move_file_name}"
movefile = files.upload_file(workflow_A_name,upload_path=f"{move_file_path}",conflict_action="MERGE").json()


POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'path': '/Workspaces/Public/move_file_example.yxmd', 'conflict_action': 'MERGE', 'Cookie': 'ayxSession=s%3A5b8d4316-2840-4aa4-a375-d7c224444acc.PnmVo6YILudIgbnDRNuZAs%2F84eXXL1f16IIR7e9f4pU', 'Content-Length': '3250'}
Response Status: 200


In [ ]:
response = files.move_file(f"{move_file_path}",r"/Workspaces/Public/move_file_destination.yxmd")

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/moveFiles
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A5b8d4316-2840-4aa4-a375-d7c224444acc.PnmVo6YILudIgbnDRNuZAs%2F84eXXL1f16IIR7e9f4pU', 'Content-Length': '197'}
Response Status: 200


In [ ]:
response.json()

[{'assetPath': '/Workspaces/Public/move_file_example.yxmd',
  'successMoved': [{'version': 1,
    'folderType': None,
    'targetId': 'ba550013-d96c-42d7-a72d-5b3ba226a509',
    'sourceId': 'ba550013-d96c-42d7-a72d-5b3ba226a509',
    'involvedVersions': 1,
    'isMergeOperation': False,
    'sourcePath': '/Workspaces/Public/move_file_example.yxmd',
    'targetPath': '/Workspaces/Public/move_file_destination.yxmd'}]}]

### Simple copy example

Now, lets try copying a file.

In [ ]:
#hide
#create an initial file to copy
move_file_name="move_file_example.yxmd"
move_file_path=f"/Workspaces/Public/{move_file_name}"
movefile = files.upload_file(workflow_A_name,upload_path=f"{move_file_path}",conflict_action="MERGE").json()


POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'path': '/Workspaces/Public/move_file_example.yxmd', 'conflict_action': 'MERGE', 'Cookie': 'ayxSession=s%3A5b8d4316-2840-4aa4-a375-d7c224444acc.PnmVo6YILudIgbnDRNuZAs%2F84eXXL1f16IIR7e9f4pU', 'Content-Length': '3250'}
Response Status: 200


In [ ]:
response = files.move_file(move_file_path,r"/Workspaces/Public/copied_file.yxmd","copy")

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/copy
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A5b8d4316-2840-4aa4-a375-d7c224444acc.PnmVo6YILudIgbnDRNuZAs%2F84eXXL1f16IIR7e9f4pU', 'Content-Length': '187'}
Response Status: 200


In [ ]:
response.json()

[{'assetPath': '/Workspaces/Public/move_file_example.yxmd',
  'successCopied': [{'targetPath': '/Workspaces/Public/copied_file.yxmd',
    'sourcePath': '/Workspaces/Public/move_file_example.yxmd',
    'version': 1,
    'folderType': None,
    'sourceId': 'adfecbb7-e210-4fba-8975-b0f1713b422f',
    'involvedVersions': 1,
    'isMergeOperation': False,
    'targetId': 'fbe53e7e-d68d-4d78-a558-e282ebcba9b8'}]}]

as you can see, in the response, there is a successCopied json object which contains information on the new file, including the targetId (the uuid of the new file).

In [ ]:
from nbdev.showdoc import *
show_doc(Files.restore_deleted_file)

<h4 id="Files.restore_deleted_file" class="doc_header"><code>Files.restore_deleted_file</code><a href="__main__.py#L94" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.restore_deleted_file</code>(**`asset_path`**:`str`=*`None`*, **`asset_id`**:`str`=*`None`*)



The restore_deleted_file method restores a specified asset. it requires either an asset_path or an asset_id. If both are specified, the asset_id is used. This can only be used to restore assets which were not hard deleted

**Arguments**:
- asset_path: (Optional) the 'trashPath' to the deleted asset including filename
- asset_id: (Optional) the UUID of the deleted asset

In [ ]:
#hide
#create an initial file to delete and delete it
delete_file_name="delete_file_example.yxmd"
delete_file_path=f"/Workspaces/Public/{delete_file_name}"
deletefile = files.upload_file(workflow_A_name,upload_path=f"{delete_file_path}",conflict_action="MERGE").json()
deletedresp = files.delete_file(delete_file_path).json()
delete_file_uuid=deletedresp[0]['id']
delete_file_trashpath=deletedresp[0]['trashPath']

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'path': '/Workspaces/Public/delete_file_example.yxmd', 'conflict_action': 'MERGE', 'Cookie': 'ayxSession=s%3A734318b6-ccc9-4c4e-a293-c8335d113121.k%2BmIPQ6vHeUa6TsiMjKijXXrRVvSm1QYfnap6GI7Jak', 'Content-Length': '3250'}
Response Status: 200
POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/softDelete
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A734318b6-ccc9-4c4e-a293-c8335d113121.k%2BmIPQ6vHeUa6TsiMjKijXXrRVvSm1QYfnap6GI7Jak', 'Content-Length': '63'}
Response Status: 200


### Restore deleted file by uuid

here we will restore a deleted file by pointing to a uuid

In [ ]:
response = files.restore_deleted_file(asset_id=delete_file_uuid)

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/restoreDeleted
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A734318b6-ccc9-4c4e-a293-c8335d113121.k%2BmIPQ6vHeUa6TsiMjKijXXrRVvSm1QYfnap6GI7Jak', 'Content-Length': '54'}
Response Status: 200


In [ ]:
response.json()

{'failed': [],
 'restored': [{'assetPath': '/Trash/delete_file_example28729.yxmd',
   'successRestored': [{'trashPath': '/Trash/delete_file_example28729.yxmd',
     'assetId': 'fe5156c4-bf12-4929-9a32-c4f1da41693d',
     'folderType': None,
     'version': 1,
     'restorePath': '/Workspaces/Public/delete_file_example.yxmd'}]}]}

### Restore deleted file by asset_path

here we will restore a deleted file using the 'trashPath' provided when an asset is deleted.

In [ ]:
response = files.restore_deleted_file(asset_path=delete_file_trashpath)

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/restoreDeleted
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A734318b6-ccc9-4c4e-a293-c8335d113121.k%2BmIPQ6vHeUa6TsiMjKijXXrRVvSm1QYfnap6GI7Jak', 'Content-Length': '56'}
Response Status: 200


In [ ]:
response.json()

{'failed': [],
 'restored': [{'assetPath': '/Trash/delete_file_example28929.yxmd',
   'successRestored': [{'trashPath': '/Trash/delete_file_example28929.yxmd',
     'assetId': 'b0d82f4c-634b-4ce3-b889-d386beb696fb',
     'folderType': None,
     'version': 1,
     'restorePath': '/Workspaces/Public/delete_file_example.yxmd'}]}]}